In [1]:
import pandas as pd

In [2]:
#  This warning will not affect the results for what I am testing
data = pd.read_csv("../data/ICPSR_36371/DS0001/36371-0001-Data.tsv", sep='\t')
for col in data:
    data[col] = pd.to_numeric(data[col], errors='coerce').fillna(-1).astype(int)

/var/folders/sm/xf3_gccn5351gstdytg6dvsc0000gn/T/ipykernel_90732/3045689864.py:2: DtypeWarning: Columns (19,24,25,43,45,46,47,48,49,50,51,52,53,216,217,218,221,223,224,226,227,228,229,239,240,241,242,243,244,292,308,309,494,495,496,497,509) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../data/ICPSR_36371/DS0001/36371-0001-Data.tsv", sep='\t')


In the code below I am classifying which repondents were surveyed in each wave. You will see that many respondents have been interviewed in multiple waves. As I see it we have three major options, (1)treat each wave of responses as a seperate interview (even if the respondent is the same person), (2) do nothing, or (3) only use 1 wave of responses for each respondent.

(1) In the first case, we would create a new row each time a respondent has been interviewed. Meaning if there is data for respondent X for Wave 1 and 2, we would have sample X1 and sample X2. While we gain samples, I am hesitant about this approach since we are using highly correlated data for different samples. In this case we would have to 'squeeze' attributes. For example there are the following varibles for each sample W1MALE, W2MALE, and W3MALE. Since we would only be considering one wave for each sample, we would need to convert these three varaibles to one generic varaible MALE.

(2) Doing nothing may be the best option since we can say that our data is robust. However, I am concerned about the sparcity of our dataset. Going back to the W1MALE, W2MALE, and W3MALE example, we have many variables that are currently held across three columns corresponding to each wave. My concern in the current form is that we have unnessary sparcity that we could decrease.

(3) Although we will incure a loss in data, I think we should likely only consider one set of responses from each respondent. While there are many different ways we can decide which wave to choose when a respondent has responded to more than one wave of interviews, I think the major advantage is the compression of columns. Since we are only considering one wave per respondent, we can avoid the W1MALE, W2MALE, and W3MALE problem, and condense these variables into one MALE varible.

In [4]:
sample_wave = [0 for i in range(3661)]

for i in range(3661):

    # 1, 2, 3
    if data["PANEL123"][i] == 1 and data["ALLWAV1"][i] == 1 and data["ALLWAV2"][i] == 1 and data["ALLWAV3"][i] == 1:
        sample_wave[i] = 123

    # 1 and 3
    elif data["PANEL103"][i] == 1 and data["ALLWAV1"][i] == 1 and data["ALLWAV3"][i] == 1:
        sample_wave[i] = 13

    # 1 and 2
    elif data["PANEL120"][i] == 1 and data["ALLWAV1"][i] == 1 and data["ALLWAV2"][i] == 1:
        sample_wave[i] = 12

    # 2 and 3
    elif data["PANEL023"][i] == 1 and data["ALLWAV2"][i] == 1 and data["ALLWAV3"][i] == 1:
        sample_wave[i] = 23
    
    # 1
    elif data["ALLWAV1"][i] == 1:
        sample_wave[i] = 1

    # 2
    elif data["ALLWAV2"][i] == 1:
        sample_wave[i] = 2
    
    # 3
    elif data["ALLWAV3"][i] == 1:
        sample_wave[i] = 3
    

print("Samples in Waves 1, 2, and 3:", sample_wave.count(123))
print()
print("Samples in Waves 1 and 2:", sample_wave.count(12))
print("Samples in Waves 1 and 3:", sample_wave.count(13))
print("Samples in Waves 2 and 3:", sample_wave.count(23))
print()
print("Samples in Wave 1:", sample_wave.count(1))
print("Samples in Wave 2:", sample_wave.count(2))
print("Samples in Wave 3:", sample_wave.count(3))
print()
print("Check 1 (1,2):", sample_wave.count(12) + sample_wave.count(123), "= 679")
print("Check 2 (2,3):", sample_wave.count(23) + sample_wave.count(123), "= 818")
print("Check 3 (1,2,3):", sample_wave.count(123), "= 349")
print("Check 4 (1,3):", sample_wave.count(13), "= 11")
print("Check 5 (Totals): 3361 =", sample_wave.count(1) + sample_wave.count(2) + sample_wave.count(3) + sample_wave.count(12) + sample_wave.count(13) + sample_wave.count(23) + sample_wave.count(123))
print("Check 6 (Unnacounted For):", sample_wave.count(-1) ,"= 0")

Samples in Waves 1, 2, and 3: 349

Samples in Waves 1 and 2: 330
Samples in Waves 1 and 3: 11
Samples in Waves 2 and 3: 469

Samples in Wave 1: 760
Samples in Wave 2: 815
Samples in Wave 3: 927

Check 1 (1,2): 679 = 679
Check 2 (2,3): 818 = 818
Check 3 (1,2,3): 349 = 349
Check 4 (1,3): 11 = 11
Check 5 (Totals): 3361 = 3661
Check 6 (Unnacounted For): 0 = 0
